In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os



In [ ]:
data_path = os.path.join("..", "data")

In [ ]:
X = np.load(data_path + '/initial_data/function_4/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_4/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.428571,0.448979,0.387755,0.448979]], axis=0)
y = np.append(y, [-0.281966339], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.469388, 0.469388, 0.285714, 0.428571]], axis=0)
y = np.append(y, [-2.5093632338739513], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.372475, 0.484827, 0.477273, 0.411341]], axis=0)
y = np.append(y, [-2.2686705479562543], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.310497, 0.337378, 0.332006, 0.378659]], axis=0)
y = np.append(y, [-0.8114645399106837], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.437681, 0.304730, 0.402971, 0.465417]], axis=0)
y = np.append(y, [-1.5478681188544452], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.296830, 0.339701, 0.328080, 0.381113]], axis=0)
y = np.append(y, [-1.2401877238019376], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.409933, 0.364803, 0.392926, 0.352123]], axis=0)
y = np.append(y, [0.5774975942545706], axis=0)

# add the week 8 input and result to the array
X = np.append(X, [[0.402280, 0.380074, 0.375910, 0.363568]], axis=0)
y = np.append(y, [0.388762689545207], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
print(f"max y is :{np.max(y)}")

In [ ]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

In [ ]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
# week 7 changed kappa to 1.0 from 4.0
# week 8 changed kappa to 4.0.5 from §.0 for more exploitation
def ucb(x, gp, kappa=0.5):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
"""grid_size = 50
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
x3 = np.linspace(0, 1, grid_size)
x4 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b, c, d] for a in x1 for b in x2 for c in x3 for d in x4])"""

In [ ]:
n_samples = 3000000 # You can adjust this based on your compute budget

# Generate random 6D points in [0, 1]^6
X_random = np.random.rand(n_samples, 4)

In [ ]:
ucb_values = np.array([ucb(x, gp) for x in X_random])


In [ ]:
best_idx = np.argmax(ucb_values)
next_best_point = X_random[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Next best point to evaluate (UCB) rounded:", np.round(next_best_point,6))    
print("Next best point to evaluate (UCB) without rounding:", next_best_point)  
print("Predicted maximum value (mean):", pred_mean[0])




In [ ]:
# SVM model as surrogate
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 4)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)